In [28]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [29]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [30]:
llm.invoke("How will the weather be in munich today?")

AIMessage(content='I am an AI assistant and I do not have real-time information. I recommend checking a reliable weather website or app for the most up-to-date weather forecast in Munich.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 17, 'total_tokens': 52, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8cfb8102-9867-4d66-a648-f16936a26411-0', usage_metadata={'input_tokens': 17, 'output_tokens': 35, 'total_tokens': 52, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [31]:
from langchain_core.tools import tool


@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["munich"]:
        return "It's 15 degrees Celsius and cloudy."
    else:
        return "It's 32 degrees Celsius and sunny."


@tool
def check_seating_availability(location: str, seating_type: str):
    """Call to check seating availability."""
    if location.lower() == "munich" and seating_type.lower() == "outdoor":
        return "Yes, we still have seats available outdoors."
    elif location.lower() == "munich" and seating_type.lower() == "indoor":
        return "Yes, we have indoor seating available."
    else:
        return "Sorry, seating information for this location is unavailable."


tools = [get_weather, check_seating_availability]

In [42]:
get_weather.invoke({'location': 'munich'})

"It's 15 degrees Celsius and cloudy."

In [44]:
check_seating_availability({'location': 'munich', 'seating_type': 'outdoor'})

'Yes, we still have seats available outdoors.'

In [45]:
llm_with_tools = llm.bind_tools(tools)

In [46]:
result = llm_with_tools.invoke("How will the weather be in munich today?")
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_UnxghesCCoNxAmHkfwqaU9rl', 'function': {'arguments': '{"location":"Munich"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 84, 'total_tokens': 101, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-075dd5c3-659d-4298-8adf-d67ab0bef84b-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Munich'}, 'id': 'call_UnxghesCCoNxAmHkfwqaU9rl', 'type': 'tool_call'}], usage_metadata={'input_tokens': 84, 'output_tokens': 17, 'total_tokens': 101, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [47]:
result.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'Munich'},
  'id': 'call_UnxghesCCoNxAmHkfwqaU9rl',
  'type': 'tool_call'}]

In [48]:
result = llm_with_tools.invoke(
    "How will the weather be in munich today? Do you still have seats outdoor available?"
)
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_RcdiCTuq0vYi4iRGa2tsA2e1', 'function': {'arguments': '{"location": "munich"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_qR5tUtgLHEQ7TMgF7TYqsTyv', 'function': {'arguments': '{"location": "munich", "seating_type": "outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 92, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-154bb1d6-9951-4731-96db-c37b0db59659-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'munich'}, 'id': 'call_RcdiCTuq0vYi4iRGa2tsA2e1', 'type': 'tool_call'}, {'name': 

In [49]:
result.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'munich'},
  'id': 'call_RcdiCTuq0vYi4iRGa2tsA2e1',
  'type': 'tool_call'},
 {'name': 'check_seating_availability',
  'args': {'location': 'munich', 'seating_type': 'outdoor'},
  'id': 'call_qR5tUtgLHEQ7TMgF7TYqsTyv',
  'type': 'tool_call'}]

In [60]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    HumanMessage(
        "How will the weather be in munich today? Do you still have seats outdoor available?"
    )
]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [61]:
llm_output

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_CFeCr1ahUL3Z0jRzoDucuqcV', 'function': {'arguments': '{"location": "munich"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_U3MFXdJybvPd4a4jZbcrifCR', 'function': {'arguments': '{"location": "munich", "seating_type": "outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 92, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-94d430ff-bea7-4ff7-b244-7767b4dac732-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'munich'}, 'id': 'call_CFeCr1ahUL3Z0jRzoDucuqcV', 'type': 'tool_call'}, {'name': 

In [52]:
llm_output.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'munich'},
  'id': 'call_dNbmqfftBLblYw5xdL35bOEp',
  'type': 'tool_call'},
 {'name': 'check_seating_availability',
  'args': {'location': 'munich', 'seating_type': 'outdoor'},
  'id': 'call_J2lv9QoF9ytkimWfwoxv2W2R',
  'type': 'tool_call'}]

In [51]:
messages

[HumanMessage(content='How will the weather be in munich today? Do you still have seats outdoor available?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_dNbmqfftBLblYw5xdL35bOEp', 'function': {'arguments': '{"location": "munich"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_J2lv9QoF9ytkimWfwoxv2W2R', 'function': {'arguments': '{"location": "munich", "seating_type": "outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 92, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ea132eae-72d5-4d3d-b68c-

In [54]:
tool_mapping = {
    "get_weather": get_weather,
    "check_seating_availability": check_seating_availability,
}

In [56]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    print(tool)
    print("=============")
    tool_output = tool.invoke(tool_call["args"])
    print(tool_output)
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

name='get_weather' description='Call to get the current weather.' args_schema=<class 'langchain_core.utils.pydantic.get_weather'> func=<function get_weather at 0x00000157BFAB90D0>
It's 15 degrees Celsius and cloudy.
name='check_seating_availability' description='Call to check seating availability.' args_schema=<class 'langchain_core.utils.pydantic.check_seating_availability'> func=<function check_seating_availability at 0x00000157BFAF0700>
Yes, we still have seats available outdoors.


In [57]:
messages

[HumanMessage(content='How will the weather be in munich today? Do you still have seats outdoor available?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_dNbmqfftBLblYw5xdL35bOEp', 'function': {'arguments': '{"location": "munich"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_J2lv9QoF9ytkimWfwoxv2W2R', 'function': {'arguments': '{"location": "munich", "seating_type": "outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 92, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ea132eae-72d5-4d3d-b68c-

In [59]:
llm_with_tools.invoke(messages)

AIMessage(content='The weather in Munich today is 15 degrees Celsius and cloudy. Additionally, there are still seats available outdoors.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 177, 'total_tokens': 201, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e47f64ab-a5a7-42c5-8cfd-dae5be3187a1-0', usage_metadata={'input_tokens': 177, 'output_tokens': 24, 'total_tokens': 201, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [27]:
messages

[HumanMessage(content='How will the weather be in munich today? Do you still have seats outdoor available?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wMfS5pHFRY4z8dZNSJS4gDX7', 'function': {'arguments': '{"location": "munich"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_ueeSWZ0qEtBUNnxXAG1HOQcQ', 'function': {'arguments': '{"location": "munich", "seating_type": "outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 92, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-016c0cd3-e6dc-4fd2-93ca-

In [65]:
# Step 1: Manually execute tools (skipping LLM's first call)
weather_response = get_weather.invoke({"location": "munich"})
seating_response = check_seating_availability.invoke({"location": "munich", "seating_type": "outdoor"})

In [66]:
weather_response

"It's 15 degrees Celsius and cloudy."